In [1]:
import sys
sys.path.append("..")
sys.path.append("../model")
from trajcl import TrajCL
from config import Config

In [2]:
conf = Config()

conf.dataset = 'nyc'
conf.post_value_updates()

In [3]:
model = TrajCL()

/home/sagemaker-user/.conda/envs/trajcl/lib/python3.12/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [4]:
model

TrajCL(
  (clmodel): MoCo(
    (criterion): CrossEntropyLoss()
    (encoder_q): DualSTBWithTime(
      (pos_encoder): TimeBasedPosEncoding(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (structural_attn): TransformerEncoder(
        (layers): ModuleList(
          (0-1): 2 x TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
            )
            (linear1): Linear(in_features=256, out_features=2048, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (linear2): Linear(in_features=2048, out_features=256, bias=True)
            (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (dropout1): Dropout(p=0.1, inplace=False)
            (dropout2): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (spatial_attn): Sp

In [5]:
import pandas as pd
test_df = pd.read_parquet("/home/sagemaker-user/TrajCL/data/parquet_files/test/nyc_df_v3_with_time/traj_test_df_v3_with_ts.parquet")

In [6]:
userids = test_df['userid'].unique()

In [7]:
import torch
device = torch.device("cuda:0")
checkpoint_file = "/home/sagemaker-user/TrajCL/exp/v2.1/nyc_TrajCL_best.pt"
checkpoint = torch.load(checkpoint_file)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)

/tmp/ipykernel_12411/148144967.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_file)


TrajCL(
  (clmodel): MoCo(
    (criterion): CrossEntropyLoss()
    (encoder_q): DualSTBWithTime(
      (pos_encoder): TimeBasedPosEncoding(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (structural_attn): TransformerEncoder(
        (layers): ModuleList(
          (0-1): 2 x TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
            )
            (linear1): Linear(in_features=256, out_features=2048, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (linear2): Linear(in_features=2048, out_features=256, bias=True)
            (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (dropout1): Dropout(p=0.1, inplace=False)
            (dropout2): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (spatial_attn): Sp

In [9]:
from utils.traj import *
import pickle

from torch.nn.utils.rnn import pad_sequence
embs = pickle.load(open("/home/sagemaker-user/TrajCL/data/nyc_cell250_embdim256_embs.pkl", 'rb')).to('cpu').detach() # tensor
cellspace = pickle.load(open("/home/sagemaker-user/TrajCL/data/nyc_cell250_cellspace.pkl", 'rb'))

def infer(traj, time_indices):
    traj_cell, traj_p = zip(*[merc2cell2(t, cellspace) for t in traj])
    traj_emb_p = [torch.tensor(generate_spatial_features(t, cellspace)) for t in traj_p]
    traj_emb_p = pad_sequence(traj_emb_p, batch_first = False).to(device)
    traj_emb_cell = [embs[list(t)] for t in traj_cell]
    traj_emb_cell = pad_sequence(traj_emb_cell, batch_first = False).to(device)
    traj_len = torch.tensor(list(map(len, traj_cell)), dtype = torch.long, device = device)
    time_indices = pad_sequence([torch.tensor(t, dtype=torch.long) for t in time_indices], batch_first=False, padding_value=-1).to(Config.device)
    # print(traj_emb_cell, traj_emb_p, traj_len)
    traj_embs = model.interpret(traj_emb_cell.float(), traj_emb_p.float(), traj_len, time_indices)
    return traj_embs



/home/sagemaker-user/.conda/envs/trajcl/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [10]:
from sklearn.metrics.pairwise import cosine_similarity


In [11]:
test_df.head()

,userid,traj_id,timestamps_filtered,wgs_seq_filtered,merc_seq_filtered,trajlen,last_emp_start_date,traj_date,min_date,max_date,traj_count,last_pck_date,prev_pck_date,train_test_tag,paycheck_amount,label,time_index_list
0,2908166,2908166_2023-07-10,"[2023-07-10T05:58:48.000000, 2023-07-10T06:08:...","[[-74.13615838274143, 40.780548095703125], [-7...","[[-8252799.400536256, 4980026.246496455], [-82...",46,2021-07-23,2023-07-10,2021-07-23,2025-07-23,1086,2025-07-25,2025-07-18,train,826,1,"[35, 36, 38, 39, 40, 41, 43, 44, 45, 50, 51, 5..."
1,2908166,2908166_2024-04-17,"[2024-04-17T05:27:52.000000, 2024-04-17T05:42:...","[[-74.13858887961788, 40.7818603515625], [-74....","[[-8253069.962210918, 4980219.164941694], [-82...",51,2021-07-23,2024-04-17,2021-07-23,2025-07-23,1086,2025-07-25,2025-07-18,train,826,1,"[32, 34, 38, 39, 40, 42, 44, 45, 46, 47, 48, 5..."
2,2908166,2908166_2024-12-29,"[2024-12-29T04:37:54.000000, 2024-12-29T04:50:...","[[-74.13963465958288, 40.78245892193241], [-74...","[[-8253186.377904102, 4980307.163730451], [-82...",45,2021-07-23,2024-12-29,2021-07-23,2025-07-23,1086,2025-07-25,2025-07-18,train,1318,1,"[27, 29, 30, 31, 32, 36, 37, 38, 39, 40, 41, 4..."
3,2908166,2908166_2025-03-01,"[2025-03-01T04:50:35.000000, 2025-03-01T09:54:...","[[-74.13176415310585, 40.77603776627111], [-74...","[[-8252310.237130794, 4979363.199270785], [-82...",55,2021-07-23,2025-03-01,2021-07-23,2025-07-23,1086,2025-07-25,2025-07-18,train,839,1,"[29, 59, 70, 75, 76, 77, 78, 79, 80, 81, 82, 8..."
4,2908166,2908166_2022-10-14,"[2022-10-14T04:32:56.000000, 2022-10-14T04:45:...","[[-74.14570463447282, 40.78564453125], [-74.14...","[[-8253862.084417979, 4980775.509244509], [-82...",58,2021-07-23,2022-10-14,2021-07-23,2025-07-23,1086,2025-07-25,2025-07-18,train,826,1,"[27, 28, 29, 30, 31, 32, 33, 35, 36, 38, 39, 4..."


In [13]:
test_df['time_index_list'].values[0]

array([ 35,  36,  38,  39,  40,  41,  43,  44,  45,  50,  51,  52,  53,
        55,  56,  58,  59,  60,  80,  85,  86,  87,  88,  90,  91,  92,
        93,  94,  95,  97,  98, 100, 101, 105, 106, 107, 109, 110, 111,
       113, 114, 115, 116, 118, 119, 120], dtype=int32)

In [ ]:
from tqdm import tqdm
model.eval()
gt_list = []
pred_list = []
def get_gt_and_pred_label(userid):
    user_data = test_df[test_df['userid'] == userid].reset_index(drop=True)
    train_data = user_data[user_data['train_test_tag'] == 'train'].reset_index(drop=True)
    test_data = user_data[user_data['train_test_tag'] == 'test'].reset_index(drop=True)
    train_traj = train_data['merc_seq_filtered'].values
    test_traj = test_data['merc_seq_filtered'].values
    train_time_indices = train_data['time_index_list'].values
    test_time_indices = test_data['time_index_list'].values
    train_embs = infer(train_traj, train_time_indices).detach().cpu().numpy()
    test_embs = infer(test_traj, test_time_indices).detach().cpu()
    if sum(test_data['paycheck_amount'].values) > 0:
        gt_label = 1
    else:
        gt_label = 0
    pred_label = 0
    for i in range(len(test_embs)):
        test_vector = test_embs[i].unsqueeze(0)
        similarity = cosine_similarity(test_vector.numpy(), train_embs)[0]
        top_3_indices = np.argsort(similarity)[-3:][::-1]
        # print(i, top_3_indices)
        similarity = similarity[top_3_indices]
        # print(f"User: {userid}, Test Trajectory {test_data['traj_id'].values[i]}, Top 3 Train Trajectories: {train_data['traj_id'].values[top_3_indices]}, similarity: {similarity}, PCK Amount: {train_data['paycheck_amount'].values[top_3_indices]}")
        for sim, idx in zip(similarity, top_3_indices):
            if sim>0.85 and train_data['paycheck_amount'].values[idx]>0:
                pred_label = 1
                break
    return gt_label, pred_label

for userid in tqdm(userids):
    gt_label, pred_label = get_gt_and_pred_label(userid)
    gt_list.append(gt_label)
    pred_list.append(pred_label)
        


In [ ]:
# % of 0 labels
sum(gt_list) / len(gt_list), sum(pred_list) / len(pred_list)

In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(gt_list, pred_list)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix

# Calculate confusion matrix
cm = confusion_matrix(gt_list, pred_list)
# Assuming you already have the confusion matrix 'cm'
# For binary classification, cm is in the form:
# [[TN, FP],
#  [FN, TP]]

tn, fp, fn, tp = cm.ravel()
print("True Positives:", tp)
print("False Positives:", fp)
print("True Negatives:", tn)
print("False Negatives:", fn)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Calculate precision and recall
precision = precision_score(gt_list, pred_list)
recall = recall_score(gt_list, pred_list)
f1 = f1_score(gt_list, pred_list)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [ ]:
def get_gt_and_pred_label(userid):
    user_data = test_df[test_df['userid'] == userid].reset_index(drop=True)
    train_data = user_data[user_data['train_test_tag'] == 'train'].reset_index(drop=True)
    test_data = user_data[user_data['train_test_tag'] == 'test'].reset_index(drop=True)
    train_traj = train_data['merc_seq_filtered'].values
    test_traj = test_data['merc_seq_filtered'].values
    train_embs = infer(train_traj).detach().cpu().numpy()
    test_embs = infer(test_traj).detach().cpu()
    if sum(test_data['paycheck_amount'].values) > 0:
        gt_label = 1
    else:
        gt_label = 0
    pred_label = 0
    for i in range(len(test_embs)):
        test_vector = test_embs[i].unsqueeze(0)
        similarity = cosine_similarity(test_vector.numpy(), train_embs)[0]
        top_3_indices = np.argsort(similarity)[-3:][::-1]
        # print(i, top_3_indices)
        similarity = similarity[top_3_indices]
        print(f"User: {userid}, Test Trajectory {test_data['traj_id'].values[i]}, Top 3 Train Trajectories: {train_data['traj_id'].values[top_3_indices]}, similarity: {similarity}, PCK Amount: {train_data['paycheck_amount'].values[top_3_indices]}")
        for sim, idx in zip(similarity, top_3_indices):
            if sim>0.85 and train_data['paycheck_amount'].values[idx]>0:
                pred_label = 1
                break
    return gt_label, pred_label

In [ ]:
correct_indices = [i for i, (gt, pred) in enumerate(zip(gt_list, pred_list)) if gt == pred]
print("Correct Indices:", correct_indices)

In [ ]:
fn_indices = [i for i, (gt, pred) in enumerate(zip(gt_list, pred_list)) if gt == 1 and pred == 0]
print("False Negative Indices:", fn_indices)

In [ ]:
fp_indices = [i for i, (gt, pred) in enumerate(zip(gt_list, pred_list)) if gt == 0 and pred == 1]
print("False Positive Indices:", fp_indices)

In [ ]:
idx = 4
userid = userids[idx]
get_gt_and_pred_label(userid)

In [ ]:
len(test_df)

In [ ]:
len(test_df[test_df['train_test_tag'] == 'test'][test_df['label']==1]['userid'].unique())

In [ ]:
790/1051

In [ ]:
test_df[test_df['userid']==5499415].sort_values('traj_date',ascending=False).head(50)